<a href="https://colab.research.google.com/github/Lilaven/Testing/blob/main/Bayesian_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code chạy predict trước
Sau đó chạy Bayesian optimization để hyperparameter tuning cho learning rate, epochs, số layers, hidden units

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import necessary packages

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
from sklearn import preprocessing

%matplotlib inline
import matplotlib.pyplot as plt



from scipy.stats import norm, loguniform
from ax.utils.tutorials.cnn_utils import load_mnist, train, evaluate, CNN
from warnings import catch_warnings
from warnings import simplefilter
from matplotlib import pyplot
from typing import Dict
from torch.utils.data import DataLoader
from tqdm import tqdm

Read excel file by Pandas

In [ ]:
import os
folder_path= "/content/drive/My Drive/MathforAI"

def filepath(file_name):
  fname = '{}.xlsx'.format(file_name)
  fpath = os.path.join(folder_path, fname)
  return(fpath)

df = pd.read_excel(filepath('DG_PROJECT_All'))
df.head()

In [ ]:
import os
folder_path= "/content/drive/My Drive/MathforAI"

def filepath(file_name):
  fname = '{}.csv'.format(file_name)
  fpath = os.path.join(folder_path, fname)
  return(fpath)

df = pd.read_csv(filepath('datapj'), encoding='utf-8', sep=',')
df.head()

,CONTRACT_NO,VINTAGE,LOAN_AMT_WITH_INS,INTEREST_RATE,TENURE,PRODUCT,AGE,GENDER,MARITAL_STATUS,SOCIAL_STATUS,EDUCATION,POS_PROVINCE,PROVINCE_CITY_CURRENT,FB_MEMBERS,PERSONAL_INCOME,DEBT_GROUP
0,20171110-0002628,201801,7339500,45.0,12,CDL,27,MALE,SINGLE,BLUE-COLLAR,2,101.47,10330.15,1,7000000,3.0
1,20171211-7916965,201801,6605550,55.0,12,CDL,48,MALE,MARRIED,WHITE-COLLAR,3,101.47,10330.15,3,7000000,2.0
2,20171221-0003360,201801,44200000,45.0,12,TW,41,MALE,SINGLE,BLUE-COLLAR,3,101.47,10330.15,5,13000000,1.0
3,20171204-0004175,201801,16880000,53.0,24,XSELL,38,MALE,MARRIED,BLUE-COLLAR,3,101.47,8462.58,3,6000000,3.0
4,20171212-0001990,201801,10550000,47.0,24,XSELL,31,MALE,SINGLE,BLUE-COLLAR,3,101.47,8462.58,4,5000000,3.0


Drop unnecesary column

In [ ]:
df.drop(columns=['CONTRACT_NO'] )

,VINTAGE,LOAN_AMT_WITH_INS,INTEREST_RATE,TENURE,PRODUCT,AGE,GENDER,MARITAL_STATUS,SOCIAL_STATUS,EDUCATION,POS_PROVINCE,PROVINCE_CITY_CURRENT,FB_MEMBERS,PERSONAL_INCOME,DEBT_GROUP
0,201801,7339500,45.00,12,CDL,27,MALE,SINGLE,BLUE-COLLAR,2,101.47,10330.15,1,7000000,3.0
1,201801,6605550,55.00,12,CDL,48,MALE,MARRIED,WHITE-COLLAR,3,101.47,10330.15,3,7000000,2.0
2,201801,44200000,45.00,12,TW,41,MALE,SINGLE,BLUE-COLLAR,3,101.47,10330.15,5,13000000,1.0
3,201801,16880000,53.00,24,XSELL,38,MALE,MARRIED,BLUE-COLLAR,3,101.47,8462.58,3,6000000,3.0
4,201801,10550000,47.00,24,XSELL,31,MALE,SINGLE,BLUE-COLLAR,3,101.47,8462.58,4,5000000,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610913,201810,28782000,21.44,18,TW,42,MALE,DIVORCED,BLUE-COLLAR,3,94.86,7650.75,5,15000000,3.0
610914,201810,21100000,47.00,18,XSELL,36,FEMALE,MARRIED,BLUE-COLLAR,3,101.47,5095.76,5,3906000,1.0
610915,201810,21100000,50.00,18,XSELL,61,MALE,MARRIED,WHITE-COLLAR,3,101.47,10330.15,4,6000000,2.0
610916,201810,45000000,43.00,24,XSELL,39,MALE,MARRIED,BLUE-COLLAR,3,101.47,5190.27,5,10000000,3.0


Transform non-numerical labels to numerical labels.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

# Find all unique value of columns need encoding
u_vintage = np.unique(df['VINTAGE'])

# Match initial label with new index encoded by LabelEncoder function
le = LabelEncoder()
label_vintage = le.fit_transform(df['VINTAGE'])
mapping_vintage = {index: label for index, label in 
                  enumerate(le.classes_)}
                  
df['VINTAGE'] = label_vintage
df['INTEREST_RATE'] = df['INTEREST_RATE']/100
df[['VINTAGE','INTEREST_RATE']]

,VINTAGE,INTEREST_RATE
0,0,0.4500
1,0,0.5500
2,0,0.4500
3,0,0.5300
4,0,0.4700
...,...,...
610913,9,0.2144
610914,9,0.4700
610915,9,0.5000
610916,9,0.4300


In [ ]:
# Find all unique value of columns need encoding
u_product= np.unique(df['PRODUCT'])
u_gender = np.unique(df['GENDER'])
u_marital = np.unique(df['MARITAL_STATUS'])
u_social = np.unique(df['SOCIAL_STATUS'])
u_debt = np.unique(df['DEBT_GROUP'])

# Match initial label with new index encoded by LabelEncoder function
le = LabelEncoder()
label_product = le.fit_transform(df['PRODUCT'])
mapping_product = {index: label for index, label in 
                  enumerate(le.classes_)}

label_gender = le.fit_transform(df['GENDER'])
mapping_gender = {index: label for index, label in 
                  enumerate(le.classes_)}

label_marital = le.fit_transform(df['MARITAL_STATUS'])
mapping_marital = {index: label for index, label in 
                  enumerate(le.classes_)}

label_social = le.fit_transform(df['SOCIAL_STATUS'])
mapping_social = {index: label for index, label in 
                  enumerate(le.classes_)}

label_debt = le.fit_transform(df['DEBT_GROUP'])
mapping_debt = {index: label for index, label in 
                  enumerate(le.classes_)}

#One hot encoding
product_onehot = pd.get_dummies(df['PRODUCT'], prefix='PRODUCT')
gender_onehot = pd.get_dummies(df['GENDER'], prefix='GENDER')
marital_onehot = pd.get_dummies(df['MARITAL_STATUS'], prefix='MARITAL_STATUS')
social_onehot = pd.get_dummies(df['SOCIAL_STATUS'], prefix='SOCIAL_STATUS')
debt_onehot = pd.get_dummies(df['DEBT_GROUP'], prefix='DEBT_GROUP')

#Concatenate one hot encoding column to dataset
endf = pd.concat([df[['VINTAGE', 'LOAN_AMT_WITH_INS', 'INTEREST_RATE','TENURE','AGE','EDUCATION','POS_PROVINCE'
                     ,'PROVINCE_CITY_CURRENT','FB_MEMBERS','PERSONAL_INCOME']]
                     , product_onehot, gender_onehot, marital_onehot, social_onehot,debt_onehot], axis=1)
endf

,VINTAGE,LOAN_AMT_WITH_INS,INTEREST_RATE,TENURE,AGE,EDUCATION,POS_PROVINCE,PROVINCE_CITY_CURRENT,FB_MEMBERS,PERSONAL_INCOME,...,SOCIAL_STATUS_PENSIONER,SOCIAL_STATUS_SELF-EMPLOYED,SOCIAL_STATUS_STUDENT,SOCIAL_STATUS_UNEMPLOYED,SOCIAL_STATUS_WHITE-COLLAR,DEBT_GROUP_1.0,DEBT_GROUP_2.0,DEBT_GROUP_3.0,DEBT_GROUP_4.0,DEBT_GROUP_5.0
0,0,7339500,0.4500,12,27,2,101.47,10330.15,1,7000000,...,0,0,0,0,0,0,0,1,0,0
1,0,6605550,0.5500,12,48,3,101.47,10330.15,3,7000000,...,0,0,0,0,1,0,1,0,0,0
2,0,44200000,0.4500,12,41,3,101.47,10330.15,5,13000000,...,0,0,0,0,0,1,0,0,0,0
3,0,16880000,0.5300,24,38,3,101.47,8462.58,3,6000000,...,0,0,0,0,0,0,0,1,0,0
4,0,10550000,0.4700,24,31,3,101.47,8462.58,4,5000000,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610913,9,28782000,0.2144,18,42,3,94.86,7650.75,5,15000000,...,0,0,0,0,0,0,0,1,0,0
610914,9,21100000,0.4700,18,36,3,101.47,5095.76,5,3906000,...,0,0,0,0,0,1,0,0,0,0
610915,9,21100000,0.5000,18,61,3,101.47,10330.15,4,6000000,...,0,0,0,0,1,0,1,0,0,0
610916,9,45000000,0.4300,24,39,3,101.47,5190.27,5,10000000,...,0,0,0,0,0,0,0,1,0,0


Natural logarithm on financial data

In [ ]:
# Save feature dataset và label to numpy array
npdf = endf.to_numpy()
data = npdf[:,0:28] 
label = npdf[:,28:33] 


In [ ]:
#Natural logarithm of financial data on training, validation and test set
def logaritdata(ds):
  ds[:,1:2] = np.log(ds[:,1:2])
  ds[:,7:8] = np.log(ds[:,7:8])
  return ds

logaritdata(data)

array([[ 0.        , 15.80878128,  0.45      , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        , 15.70342076,  0.55      , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.        , 17.60423535,  0.45      , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 9.        , 16.8647836 ,  0.5       , ...,  0.        ,
         0.        ,  1.        ],
       [ 9.        , 17.62217305,  0.43      , ...,  0.        ,
         0.        ,  0.        ],
       [ 9.        , 17.39096932,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

Split training set, validation set and test set

In [ ]:
from sklearn.model_selection import train_test_split
x, x_test, y, y_test = train_test_split(data,label, test_size=0.15, random_state=4)
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.1, random_state=4)
x_train.shape, x_valid.shape, x_test.shape, y_train.shape, y_valid.shape, y_test.shape

((467352, 28), (51928, 28), (91638, 28), (467352, 5), (51928, 5), (91638, 5))

Rescale data in some numberic data into range (0,1)

In [ ]:
def rescaledata(dataset):
  #Rescale AGE
  age = dataset[:,4].reshape(-1,1)
  sage = preprocessing.MinMaxScaler(feature_range= (0,1)).fit_transform(age)
  dataset[:,4].reshape(-1,1)
  dataset[:,4] = sage[:,0]
  #Rescale POS_PROVINCE
  pos_province = dataset[:,6].reshape(-1,1)
  spos_province = preprocessing.MinMaxScaler(feature_range= (0,1)).fit_transform(pos_province)
  dataset[:,6].reshape(-1,1)
  dataset[:,6] = spos_province[:,0]
  #Rescale TENURE
  tenure = dataset[:,3].reshape(-1,1)
  stenure = preprocessing.MinMaxScaler(feature_range= (0,1)).fit_transform(tenure)
  dataset[:,3].reshape(-1,1)
  dataset[:,3] = stenure[:,0]
  #Rescale PROVINCE_CITY_CURRENT
  prov_ct_cur = dataset[:,7].reshape(-1,1)
  sprov_ct_cur = preprocessing.MinMaxScaler(feature_range= (0,1)).fit_transform(prov_ct_cur)
  dataset[:,7].reshape(-1,1)
  dataset[:,7] = sprov_ct_cur[:,0]
  return dataset

rescaledata(x_train)
rescaledata(x_valid)
rescaledata(x_test)

x_train[:, 3:7], x_valid[:, 3:7]

(array([[0.        , 0.34782609, 3.        , 0.09298086],
        [1.        , 0.23913043, 2.        , 1.        ],
        [1.        , 0.52173913, 2.        , 1.        ],
        ...,
        [0.        , 0.04347826, 2.        , 0.49407475],
        [0.        , 0.06521739, 5.        , 1.        ],
        [0.5       , 0.86956522, 3.        , 1.        ]]),
 array([[0.        , 0.7826087 , 2.        , 1.        ],
        [0.        , 0.7826087 , 3.        , 1.        ],
        [0.5       , 0.2826087 , 5.        , 0.03099362],
        ...,
        [0.        , 0.34782609, 3.        , 0.03099362],
        [0.        , 0.41304348, 2.        , 1.        ],
        [0.        , 0.08695652, 3.        , 0.42570647]]))

Tranform numpy array object to float

In [ ]:
x_train = np.array(x_train, dtype=float)
x_valid = np.array(x_valid, dtype=float)
x_test = np.array(x_test, dtype=float)
y_train = np.array(y_train, dtype=float)
y_valid = np.array(y_valid, dtype=float)
y_test = np.array(y_test, dtype=float)

Convert numpy array to torch

In [ ]:
def convert_to_tensor(x, y):
    x_torch = torch.from_numpy(x).float()
    y_torch = torch.from_numpy(y).float()
    return x_torch, y_torch

x_train_ts, y_train_ts = convert_to_tensor(x_train, y_train)
x_valid_ts, y_valid_ts = convert_to_tensor(x_valid, y_valid)
x_test_ts, y_test_ts = convert_to_tensor(x_test, y_test)

# **Model**

In [ ]:
np.all(np.isfinite(y_train))


True

In [ ]:
np.any(np.isnan(y_train))

False

In [ ]:
# Import MLPClassifer 
from sklearn.neural_network import MLPClassifier

# Create model object
clf = MLPClassifier(hidden_layer_sizes=(6,5),
                    random_state=5,
                    verbose=True,
                    learning_rate_init=0.000001)

# Fit data onto the model
clf.fit(x_train_ts,y_train_ts)

ValueError: ignored

In [ ]:
#Model MLP

class Mymodel1(nn.Module):
    def __init__(self, input, hidden1, hidden2, output):
        super(Mymodel1, self).__init__()
        self.model = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(input, hidden1),
            nn.Tanh(),
            nn.Dropout(p=0.05),        
            nn.Linear(hidden1, hidden2),
            nn.Tanh(), 
            nn.Linear(hidden2, output),
            nn.Softmax(dim=1),
        )

    def forward(self, input):
        y_pred = self.model(input)
        return y_pred

# Training

In [ ]:
from torch.optim import Adam

mean_train_losses = []
mean_valid_losses = []
valid_acc_list = []
N_epocs, batch_size = 1000, 128
input, hidden1, hidden2, output = 28, 16, 8, 5

model = Mymodel1(input, hidden1, hidden2, output)
modelv = Mymodel1(input, hidden1, hidden2, output)
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=1e-6, eps=1e-8)

for epoch in range(N_epocs):
    model.train()
    train_losses = []
    valid_losses = []
        
    optimizer.zero_grad()
    y_pred = model(x_train_ts)
    loss = criterion(y_pred, y_train_ts)
    loss.backward()
    optimizer.step()    
    train_losses.append(loss.item())
        
    if epoch%100 == 99:
      model.eval()
      correct = 0
      total = 0
      with torch.no_grad():
        y_pred_v = model(x_valid_ts)
        loss = criterion(y_pred_v, y_valid_ts.long())
        valid_losses.append(loss.item())
        _, predicted = torch.max(y_pred_v, 1)
        _, gt = torch.max(y_valid_ts, 1)
        correct += (predicted == gt).sum().item()
        total += y_valid_ts.size(0)
        
      mean_train_losses.append(np.mean(train_losses))
      mean_valid_losses.append(np.mean(valid_losses))
      
      accuracy = 100*correct/total
      valid_acc_list.append(accuracy)
      print('epoch : {}, train loss : {:.4f}, valid loss : {:.4f}, valid acc : {:.4f}%'\
          .format(epoch+1, np.mean(train_losses), np.mean(valid_losses), accuracy))

epoch : 100, train loss : nan, valid loss : nan, valid acc : 66.0723%
epoch : 200, train loss : nan, valid loss : nan, valid acc : 66.0723%


KeyboardInterrupt: ignored

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))
ax1.plot(mean_train_losses, label='train')
ax1.plot(mean_valid_losses, label='valid')
lines, labels = ax1.get_legend_handles_labels()
ax1.legend(lines, labels, loc='best')

ax2.plot(valid_acc_list, label='valid acc')
ax2.legend()

# Acccuracy on Testdata

In [ ]:
def test_acc(model):
  correct = 0
  total = 0
  with torch.no_grad():
    for i in range(len(x_test_ts[:,0])):
      y_pred_t = model(x_test_ts)
      _, predicted = torch.max(y_pred_t, 1)
      _, gt = torch.max(y_test_ts, 1)
      correct += (predicted == gt).sum().item()
      total += y_test_ts.size(0)
  accuracy = 100*correct/total
  print('Accuracy of testdata: {:.3f}%'.format(accuracy))

test_acc(model)